Cross-entropy for stop subcategory. Only 1 alter(so not mutual cross-Entropy)

In [1]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
import seaborn as sns

In [2]:
#load of files
os.chdir("/Users/Ellen/Documents/Dokumenter/DTU/7. Semester/Bachelor")

#load data
data = pd.read_csv("dataWNames.csv")

os.chdir("/Users/Ellen/Documents/Dokumenter/DTU/7. Semester/Bachelor/CSV files")

#load probability matrix
#p = pd.read_csv("ProbAll.csv")

label = pd.read_csv("labels.csv").rename(columns={"0": 'Label'}) #load probability matrix

In [3]:
entropy = pd.read_csv("entropyBigram.csv")
entropy

,s_rand,s_unc,s_actual,diff
0,4.584963,3.073829,2.661451,-0.419526
1,3.700440,2.486614,1.971100,-0.467333
2,4.584963,2.691966,2.279918,-0.502740
3,5.044394,3.416711,2.666555,-0.471383
4,5.169925,3.594454,2.607322,-0.495675
...,...,...,...,...
92719,2.807355,1.993962,1.843522,-0.343324
92720,3.169925,1.619439,0.874660,-0.724076
92721,2.321928,1.435747,1.137616,-0.510056
92722,1.584963,1.121641,1.443609,-0.089184


In [4]:
user = data['userId'].unique().tolist()

In [5]:
label['userId'] = user

In [6]:
def contains(small, big):
    try:
        big.tostring().index(small.tostring())//big.itemsize
        return True
    except ValueError:
        return False

def cross_entropy(A, B):
    n = min(len(A), len(B))
    sum_lambda = 0

    for i in range(0, n):
        sequence = B[:i]
        
        for j in range(i+1, n+1):
            s = A[i:j]
            if contains(s, sequence) != True:
                sum_lambda += len(s)
                break
    
    h = (n*math.log2(n))/(sum_lambda)
    return h

In [7]:
#stop2 as the category
data['stop2'] = data['stop2'].astype('category')

#label encoding
data['Stop_Cat'] = data['stop2'].cat.codes

In [8]:
crossEntropy = np.zeros(len(user))
for u in user:
    n = 1
    ego = label[label.userId == u] #df with the user(ego) 
    c = ego['Label'].iloc[0] #cluster label for the user
    cluster = label[label.Label == c] #userID's for users in same cluster
    
    alters = cluster[~cluster['userId'].isin(ego['userId'].tolist())] #all in the cluster except the ego
    nAlters = alters.sample(n)['userId'].tolist() #n random alters, their userID in a list
    egoStop = data[data.userId == ego['userId'].iloc[0]]['Stop_Cat'].values
    alterStop = data[data['userId'].isin(nAlters)]['Stop_Cat'].values
    crossEntropy[user.index(u)] = cross_entropy(egoStop, alterStop)

In [9]:
crossEntropydf = pd.DataFrame(crossEntropy, columns = ['Cross-Entropy'])

In [10]:
os.chdir("/Users/Ellen/Documents/Dokumenter/DTU/7. Semester/Bachelor/CSV files")
crossEntropydf.to_csv('CrossEntropyCat.csv', index=False)